<a href="https://colab.research.google.com/github/tanongsakintean/google_colab/blob/main/mini_blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.4 MB/s eta 0:00:00


In [ ]:
import hashlib
import binascii
import datetime
import collections
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

In [ ]:
from Crypto import PublicKey




In [ ]:
class Client:
  def __init__(self):
    self.random = Crypto.Random.new().read
    self.privatekey = RSA.generate(2048,self.random)
    self.publicKey = self.privatekey.publickey()
    self.sign = PKCS1_v1_5.new(self.privatekey)
  def identity(self):
    return binascii.hexlify(self.publicKey.exportKey(format='DER')).decode('ascii')

In [ ]:


class Transaction:
  def __init__(self,sender, reciever, value):
    self.sender = sender
    self.reciever = reciever
    self.value = value
    self.time = datetime.datetime.now()

  def identity(self):
    return hashlib.sha256(str(self.to_dict()).encode()).hexdigest()

  def to_dict(self):
      if self.sender == "Genesis":
        identity = "Genesis"
      else:
        # print(self.sender.identity())
        identity = self.sender.identity()

      return collections.OrderedDict({
          'sender': identity,
          'reciever': self.reciever.identity(),
          'value': self.value,
          'time': self.time
      })


def sign_transaction(transaction, private_key):
  signer = PKCS1_v1_5.new(private_key)
  h = SHA.new(str(transaction.to_dict()).encode('utf8'))
  return binascii.hexlify(signer.sign(h)).decode('ascii')

def display_transaction(transaction):
  dict = transaction.to_dict()
  print(f"sender: {dict['sender']}")
  print(f"reciever: {dict['reciever']}")
  print(f"value: {dict['value']}")
  print(f"time: {dict['time']}")
  print("\n")



In [ ]:


class Block:
  def __init__(self):
    self.verified_transactions = []
    self.previous_block_hash = ""
    self.Nonce = ""
    self.hash = self.calculate_hash()

  def dump_blockchain(bloklist):
    print("Number of blocks in the chain: " + str(len (bloklist)))
    for x in range (len(bloklist)):
      block_temp = bloklist[x]
      print("block # " + str(x))
      for transaction in block_temp.verified_transactions:
        display_transaction(transaction)

  def calculate_hash(self):
    sha = hashlib.sha256()
    # Check if previous_block_hash or Nonce is None. If so, set them to an empty string.
    if self.previous_block_hash is None:
      self.previous_block_hash = ""
    if self.Nonce is None:
      self.Nonce = ""
    hash_str = self.previous_block_hash + str(self.Nonce) # Ensure Nonce is a string
    sha.update(hash_str.encode('utf-8'))
    return sha.hexdigest()

  def mine_block(self,difficulty):
    while self.hash[:difficulty] != '0' * difficulty:
      self.Nonce += 1
      self.hash = self.calculate_hash()

In [ ]:
transaction_list = []
block_list = []
transaction_index = 0

client1 = Client()
client2 = Client()




genesis_block = Block()
genesis_block.previous_block_hash = None
genesis_block.Nonce = None

t0 = Transaction("Genesis",client1,50)
t0.signature = sign_transaction(t0,client1.privatekey )
transaction_list.append(t0)
genesis_block.verified_transactions.append(t0)

digest = hash(genesis_block)
last_hblock = digest
genesis_block.hash = digest
block_list.append(genesis_block)


new_block = Block()
new_block.previous_block_hash = None
new_block.Nonce = None

t1 = Transaction(client2,client1,100)
t1.signature = sign_transaction(t1,client1.privatekey )
transaction_list.append(t1)
new_block.verified_transactions.append(t1)

digest = hash(new_block)
last_hblock = digest
new_block.hash = digest
block_list.append(new_block)



new_block = Block()
new_block.previous_block_hash = None
new_block.Nonce = None

t2 = Transaction(client1,client2,150)
t2.signature = sign_transaction(t2,client2.privatekey )
transaction_list.append(t2)
new_block.verified_transactions.append(t2)

digest = hash(new_block)
last_hblock = digest
new_block.hash = digest
block_list.append(new_block)



new_block = Block()
new_block.previous_block_hash = None
new_block.Nonce = None

t3 = Transaction(client2,client1,250)
t3.signature = sign_transaction(t3,client1.privatekey )
transaction_list.append(t3)
new_block.verified_transactions.append(t3)

digest = hash(new_block)
last_hblock = digest
new_block.hash = digest
block_list.append(new_block)


new_block = Block()
new_block.previous_block_hash = None
new_block.Nonce = None

t4 = Transaction(client1,client2,350)
t4.signature = sign_transaction(t4,client1.privatekey )
transaction_list.append(t4)
new_block.verified_transactions.append(t4)

digest = hash(new_block)
last_hblock = digest
new_block.hash = digest
block_list.append(new_block)

Block.dump_blockchain(block_list)



Number of blocks in the chain: 5
block # 0
sender: Genesis
reciever: 30820122300d06092a864886f70d01010105000382010f003082010a0282010100aba03a9ab8e359b72212f61ebca16c00d7e68fcad3e3a6d8e7349701cc973e47ae789702e8549cf83f2d0bd0262a781eed3149bc346d066d956e5922ee66e0c82de8b873bed435a76cc94b2eccf394124213dc3ac09498374374a6a517f4411cafff77be3d2a88fdc0dfb4eed3ab391637faaee7009912e75f05cc88d38400b6a6b225c77bfd48941bcd8082d3fdbc58c76a0235ea11cc140ca45a070c899c62b086d9b0bf8e520bf7350378b62990a4aeb1452c6f4aab40a8ef19651a571515e1dca8210d2e3c24b2f39ce28efb9267036b04cae77c571f19515955af1a6aeefb592dcbb03f948b649d023bbac6008f49a0ee3aba9f9b8f640cf8bb99ffb9950203010001
value: 50
time: 2024-09-02 08:32:43.451535


block # 1
sender: 30820122300d06092a864886f70d01010105000382010f003082010a0282010100a356ebd7065058ad0f527941e6aeda9531ebb7d871461268eab390e93aadda7ef9d0ea8332c0092fa28fd9c7b8f99ea11ac6331cbf1e468f587cecad41d47e3db463cfcc7839342e51eecafb5521680d6d3a919b65fdc64e57bd72499667e183417c292c13cedb1632b1d